# Loop and Append to Zarr Store

Assumes store has been initiated.

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
from dask.distributed import Client, progress, LocalCluster
import zarr
import glob

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:56284 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [4]:
# Tell blosc not to use threads since we are using dask to parallelize
numcodecs.blosc.use_threads = False

In [8]:
# Invariants
year = 2002
prefix = '/Volumes/Elements/netcdf'
chunks = {'lat': 500, 'lon': 500, 'time': 100}
store_dir = '/Volumes/Elements/zarr/test-500x500x5-2/'

In [9]:
# open the existing data
existing_ds = xr.open_zarr(store_dir)
existing_ds

ValueError: group not found at path ''

In [7]:
def generate_file_list(start_doy, end_doy):   
    """
    Given a start day and end end day, generate a list of file locations.
    Assumes a 'prefix' and 'year' variables have already been defined.
    'Prefix' should be a local directory or http url and path.
    'Year' should be a 4 digit year.
    """
    days_of_year = list(range(start_doy, end_doy))
    fileObjs = []

    for doy in days_of_year:
        file = glob.glob(f"{prefix}/{year}/{doy}/*.nc")[0]
        fileObjs.append(file)
    return fileObjs

## Loop and Append

In [ ]:
%%time
start_doy = 152
end_doy = start_doy
number_batches_to_append = 1
batch_size = 5
final_end_doy = start_doy + (number_batches_to_append * batch_size)

while start_doy < final_end_doy:
    end_doy = start_doy + batch_size   
    fileObjs = generate_file_list(start_doy, end_doy)
    print(fileObjs[0])
    print(f"start doy: {start_doy}, file: {fileObjs[0].split('/')[-1]}")
    print(f"end doy: {end_doy}, file: {fileObjs[-1].split('/')[-1]}")          
    ds = xr.open_mfdataset(fileObjs, chunks=chunks, parallel=True, combine='by_coords')
    ds_rechunk = ds.chunk(chunks=chunks)
    #ds_rechunk.to_zarr(store_dir, consolidated=True, mode='a', append_dim='time')
    ds_rechunk.to_zarr(store_dir, consolidated=True, mode='w')
    start_doy = end_doy
    print(f"Done with this batch")
    print()

/Volumes/Elements/netcdf/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
start doy: 152, file: 20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
end doy: 157, file: 20020605090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc


## Handling failures

In cases where the above method errors for whatever reason, we can create the subset on which the error occurred and then copy to the final datastore

In [42]:
start_doy = 177
end_doy = 182 
fileObjs = generate_file_list(start_doy, end_doy)
print(fileObjs[0])
print(f"start doy: {start_doy}, file: {fileObjs[0].split('/')[-1]}")
print(f"end doy: {end_doy}, file: {fileObjs[-1].split('/')[-1]}")          
ds = xr.open_mfdataset(fileObjs, chunks=chunks, parallel=True, combine='by_coords')
ds_rechunk = ds.chunk(chunks=chunks)
subset_source = '/Volumes/Elements/zarr/subset/'
# ds_rechunk.to_zarr(subset_source, consolidated=True, mode='w')

# existing_group = zarr.open(store=store_dir)
# subset_group = zarr.open(store=subset_source)
# zarr.copy(subset_group, existing_group, name='mursst', if_exists='replace')

# Now we test

In [32]:
start_doy = 152
end_doy = 182
fileObjs = generate_file_list(start_doy, end_doy)
print(fileObjs[0])
print(f"start doy: {start_doy}, file: {fileObjs[0].split('/')[-1]}")
print(f"end doy: {end_doy}, file: {fileObjs[-1].split('/')[-1]}")          
ds_netcdf = xr.open_mfdataset(fileObjs, chunks=chunks, parallel=True, combine='by_coords')

/Volumes/Elements/netcdf/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
start doy: 152, file: 20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
end doy: 182, file: 20020630090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc


In [39]:
%%time
ds_netcdf.analysed_sst.sel(lat=slice(10,30),lon=slice(-150,-140)).mean({'lat','lon'}).values

distributed.nanny - WARNING - Worker process 36578 was killed by signal 11
distributed.nanny - WARNING - Restarting worker


CPU times: user 2.83 s, sys: 203 ms, total: 3.04 s
Wall time: 6.85 s


array([297.46622, 297.52393, 297.5348 , 297.53952, 297.38562, 297.29898,
       297.25766, 297.25125, 297.23642, 297.21124, 297.18304, 297.16214,
       297.2236 , 297.3492 , 297.39536, 297.39502, 297.33682, 297.31046,
       297.34378, 297.35413, 297.35556, 297.35083, 297.35556, 297.3783 ,
       297.4213 , 297.49994, 297.541  , 297.51422, 297.4339 , 297.41354],
      dtype=float32)

In [40]:
%%time
existing_ds.analysed_sst.sel(lat=slice(10,30),lon=slice(-150,-140)).mean({'lat','lon'}).values

CPU times: user 589 ms, sys: 51.7 ms, total: 641 ms
Wall time: 1.5 s


array([297.46622, 297.52393, 297.5348 , 297.53952, 297.38562, 297.29898,
       297.25766, 297.25125, 297.23642, 297.21124, 297.18304, 297.16214,
       297.2236 , 297.3492 , 297.39536, 297.39502, 297.33682, 297.31046,
       297.34378, 297.35413, 297.35556, 297.35083, 297.35556, 297.3783 ,
       297.4213 , 297.49994, 297.541  , 297.51422, 297.4339 , 297.41354],
      dtype=float32)